In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

data = pd.read_csv('pseudodata_præoperation.csv')

# # #make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

X = StandardScaler().fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Create a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split data into holdout set and remaining set
indices = np.arange(len(dataset))
np.random.seed(42)
np.random.shuffle(indices)
holdout_indices = indices[:500]
remaining_indices = indices[500:]

holdout_set = Subset(dataset, holdout_indices)
remaining_set = Subset(dataset, remaining_indices)

# Define the ANN model with L2 regularization
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_dim = 575
hidden_dim = 128
output_dim = 2

# Function to train and evaluate the model with L2 regularization
def train_and_evaluate_model(train_loader, val_loader, input_dim, hidden_dim, output_dim, num_epochs=10, lr=0.01, weight_decay=1e-5):
    model = ANNModel(input_dim, hidden_dim, output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate on validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy, model

# Outer 10-Fold Cross-Validation
outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []
best_model_params = None
best_outer_accuracy = 0

for outer_train_index, outer_test_index in outer_kf.split(remaining_indices):
    outer_train_subset = Subset(remaining_set, outer_train_index)
    outer_test_subset = Subset(remaining_set, outer_test_index)
    
    # Inner 10-Fold Cross-Validation
    inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_accuracies = []

    for inner_train_index, inner_val_index in inner_kf.split(outer_train_index):
        inner_train_indices = np.array(outer_train_index)[inner_train_index]
        inner_val_indices = np.array(outer_train_index)[inner_val_index]
        
        inner_train_subset = Subset(remaining_set, inner_train_indices)
        inner_val_subset = Subset(remaining_set, inner_val_indices)
        
        train_loader = DataLoader(inner_train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(inner_val_subset, batch_size=64, shuffle=False)
        
        accuracy, model = train_and_evaluate_model(train_loader, val_loader, input_dim, hidden_dim, output_dim)
        inner_accuracies.append(accuracy)
        print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

    mean_inner_accuracy = np.mean(inner_accuracies)
    print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

    train_loader = DataLoader(outer_train_subset, batch_size=64, shuffle=True)
    test_loader = DataLoader(outer_test_subset, batch_size=64, shuffle=False)
    
    outer_accuracy, outer_model = train_and_evaluate_model(train_loader, test_loader, input_dim, hidden_dim, output_dim)
    outer_accuracies.append(outer_accuracy)
    print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

    if outer_accuracy > best_outer_accuracy:
        best_outer_accuracy = outer_accuracy
        best_model_params = outer_model.state_dict()

print(f'Mean Outer Accuracy: {np.mean(outer_accuracies) * 100:.2f}%')
print('Best model parameters:', best_model_params)

# Evaluate the final model on the holdout set
final_model = ANNModel(input_dim, hidden_dim, output_dim)
final_model.load_state_dict(best_model_params)
train_loader = DataLoader(remaining_set, batch_size=64, shuffle=True)
holdout_loader = DataLoader(holdout_set, batch_size=64, shuffle=False)

# Train the final model on the remaining set and evaluate on the holdout set
final_model_accuracy, _ = train_and_evaluate_model(train_loader, holdout_loader, input_dim, hidden_dim, output_dim)
print(f'Holdout Set Accuracy: {final_model_accuracy * 100:.2f}%')


Inner Fold Accuracy: 45.04%
Inner Fold Accuracy: 54.55%
Inner Fold Accuracy: 48.76%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 54.13%
Inner Fold Accuracy: 47.93%
Inner Fold Accuracy: 49.79%
Inner Fold Accuracy: 46.47%
Inner Fold Accuracy: 52.70%
Inner Fold Accuracy: 53.53%
Outer Fold Mean Inner Accuracy: 50.12%
Outer Fold Accuracy: 50.93%
Inner Fold Accuracy: 47.52%
Inner Fold Accuracy: 44.63%
Inner Fold Accuracy: 49.59%
Inner Fold Accuracy: 51.65%
Inner Fold Accuracy: 45.04%
Inner Fold Accuracy: 51.24%
Inner Fold Accuracy: 44.81%
Inner Fold Accuracy: 51.87%
Inner Fold Accuracy: 48.13%
Inner Fold Accuracy: 50.21%
Outer Fold Mean Inner Accuracy: 48.47%
Outer Fold Accuracy: 45.35%
Inner Fold Accuracy: 46.28%
Inner Fold Accuracy: 52.89%
Inner Fold Accuracy: 46.28%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 50.00%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 48.13%
Inner Fold Accuracy: 51.04%
Inner Fold Accuracy: 47.30%
Inner Fold Accuracy: 53.11%
Outer Fold Mean Inner Accu